In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project'
%ls
path = os.getcwd()
print('path: ' + path)
OutputPath = path + '/Output'
DataPath = path + '/Data'

/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project
 Chat_GPT_rating_accuracy.ipynb   GPT2_prediction.ipynb   Output/
 CNN_rating_prediction.ipynb      LLaVa_Inference.ipynb  'עותק של LLaVa_Inference.ipynb'
 Data/                            Models/
path: /content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project


In [3]:
import pandas as pd


In [4]:
# Paths
chatgpt_scoring_path = DataPath + '/chatGPT_scoring.txt'  # Path to the chatGPT scoring file
actual_scores_path = DataPath + '/reviews.csv'  # File containing actual scores

In [5]:
# Load review data
df = pd.read_csv(actual_scores_path)
df_of_tastes = df[df['rating_category'] == 'taste'].copy()
df_of_tastes.reset_index(drop=True, inplace=True)

In [18]:
# Extract only the filename (without path) and convert to lowercase
df_of_tastes['photo_filename'] = df_of_tastes['photo'].apply(lambda x: os.path.basename(x).lower())

In [34]:
# Function to extract image names and ratings from chatGPT scoring text
def extract_chatgpt_scores(file_path):
    with open(file_path, 'r') as file:
        data = file.read()

    # More flexible regex pattern to capture variations
    pattern = r"([A-Za-z0-9_\-\(\)\s]+\.png)[^\d]*(\d)\/5"
    matches = re.findall(pattern, data)

    chatgpt_scores = []
    for match in matches:
        image_name = match[0].strip().lower()  # Convert image name to lowercase for matching
        rating = int(match[1].strip())
        chatgpt_scores.append({'photo_filename': image_name, 'chatgpt_rating': rating})

    return pd.DataFrame(chatgpt_scores)

In [41]:
# Function to extract image names and ratings from chatGPT scoring text
def extract_chatgpt_scores(file_path):
    with open(file_path, 'r') as file:
        data = file.read()

    # Updated regex to capture variations such as ratings with extra information
    pattern = r"([A-Za-z0-9_\-\(\)\s]+\.png).*?(\d(?:\.\d)?)/5"

    chatgpt_scores = []
    unmatched_lines = []

    for line in data.split('\n'):
        # Clean up any extra spaces from the line
        line = line.strip()

        # Skip empty lines or purely descriptive lines
        if not line or not re.search(r'\d/5', line):
            continue

        match = re.search(pattern, line)
        if match:
            image_name = match.group(1).strip().lower()  # Normalize image name to lowercase
            rating = int(float(match.group(2).strip()))  # Convert decimal ratings to integer
            chatgpt_scores.append({'photo_filename': image_name, 'chatgpt_rating': rating})
        else:
            unmatched_lines.append(line)

    print(f"Unmatched lines: {unmatched_lines}")
    return pd.DataFrame(chatgpt_scores)

In [42]:
# Extract ChatGPT ratings from the file
chatgpt_scores_df = extract_chatgpt_scores(chatgpt_scoring_path)
chatgpt_scores_df.info()

Unmatched lines: ['Second Image (Abidin Tantuni Meltem Ovali) - 5/5', 'Third Image (Abidin Tantuni Ugurcan Arbaz) - 3.5/5', 'Fourth Image (Ahşap Iskender Ersin Kocabaldır) - 4/5', 'Fifth Image (Ahşap Iskender Faruk Colak) - 3.5/5', 'Sixth Image (Ahşap Iskender Sena Akim) - 4/5', 'Seventh Image (Akpınar Restaurant Ertan Yılmaz) - 3.5/5', 'Eighth Image (Akpınar Restaurant Gülcan Kaya) - 4/5', 'Ninth Image (Akpınar Restaurant Tuğçe Özkan) - 3.5/5', 'Churchill_Bistro_Bojan_Kuruc - 5/5', 'Churchill_Bistro_Sena_Sarioglu - 4/5', 'Churchill_Bistro_Sibel_Sarihan - 3/5', 'Cici_Tantuni_Ali_Sezer - 2/5', 'Cici_Tantuni_Ali_Sezer - 3/5', 'Cici_Tantuni_Alp_Tekin - 3/5', 'Cici_Tantuni_Alp_Tekin - 4/5', 'Cici_Tantuni_Mert_Tanis - 2/5', 'Cici_Tantuni_Veli_Dedeoglu - 3/5', 'Cici_Tantuni_Veli_Dedeoglu - 4/5', 'Rating: 3.5/5', 'Rating: 4.5/5', 'Rating: 3/5', 'Rating: 4/5', 'Rating: 4.5/5', 'Rating: 4/5']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 2 columns):

In [43]:
# Merge ChatGPT ratings with actual scores based on the normalized 'photo_filename' column
merged_scores = pd.merge(chatgpt_scores_df, df_of_tastes[['photo_filename', 'rating']], on='photo_filename', how='inner')

# Ensure valid comparisons by dropping rows where there are no ratings
merged_scores.dropna(subset=['chatgpt_rating', 'rating'], inplace=True)

# Calculate accuracy
if len(merged_scores) > 0:
    accuracy_chatgpt = (merged_scores['chatgpt_rating'] == merged_scores['rating']).mean() * 100
    print(f"Accuracy of ChatGPT predictions: {accuracy_chatgpt:.2f}%")
else:
    print("No valid rows to calculate accuracy.")

# Save the merged results to a CSV file (optional)
merged_scores.to_csv(OutputPath + '/chatgpt_vs_actual_scores.csv', index=False)

Accuracy of ChatGPT predictions: 23.96%
